### 생성된 학습용 데이터 가공

In [ ]:
# 필요한 라이브러리 불러오기
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.combine import SMOTEENN
from impyute.imputation.cs import mice
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
# 입력데이터 불러오기
raw_data = pd.read_csv('check5_독립변수_전처리전.csv', index_col = 0)

In [ ]:
# 이상값을 결측치로 변경 : -999 -> Nan
raw_data = raw_data.replace(-999.0, np.NaN)

# 변수별 결측치 존재여부 확인_결측제거 전
plt.figure(figsize = (12, 8))
sns.heatmap(raw_data.isnull(), cbar=True)

plt.show()

In [ ]:
# MICE를 이용하여 결측치를 대치
# 결측치가 제거된 데이터값 : eli_null
eli_null = mice(raw_data.values)

# 결측치가 제거된 데이터값을 다시 취합
in_data = pd.DataFrame(data = eli_null, columns = raw_data.columns)
in_data = in_data.clip(lower = 0)

# 변수별 결측치 존재여부 확인_결측제거 후
plt.figure(figsize = (12, 8))
sns.heatmap(in_data.isnull(), cbar=True)

plt.show()

In [ ]:
# 변수간 상관계수 확인
corr_check = in_data.corr()

# 상관관계 heatmap 생성
# annnot : 실제값 표시
# cmap : 색상
# vmin, vmax : 컬러차트 영역 설정
plt.rcParams['figure.figsize'] = (10, 10)
sns.heatmap(corr_check, annot = True, cmap = 'Pastel1_r', vmin = -1, vmax = 1)

In [ ]:
# 다중공산성 고려를 위한 변수별 VIF값 생성
vif = pd.DataFrame(data = [variance_inflation_factor(in_data.values, i) for i in range(8)], index = in_data.columns, columns = ['VIF'])
vif

In [ ]:
in_data = in_data.reset_index().rename(columns = {'index' : 'grid'})
in_data.to_csv('check6_1_입력데이터_결측제거.csv')

In [ ]:
# 이상치 제거
# 4분위수의 iqr*1.5만큼의 외부 범위를 제외한 데이터를 outlier로 간주
# 각 범위를 lbd, ubd로 계산
qut = in_data.describe().loc[['25%', '75%'], '고속도로':'주차장'].T
qut['iqr'] = qut['75%'] - qut['25%']
qut['lbd'], qut['ubd'] = qut['25%'] - 1.5 * qut['iqr'], qut['75%'] + 1.5 * qut['iqr']

# outlier 데이터는 'count'를 NaN으로 변경
for idx, srs in in_data.iterrows():
  if (qut['lbd'] < srs['고속도로':'주차장']).all() and (srs['고속도로':'주차장'] < qut['ubd']).all():
    continue
  else:
    in_data.at[idx, 'grid'] = -1
in_data

In [ ]:
# outlier 내부의 데이터만 추출
cpl_data = in_data[in_data['grid'] != -1]
cpl_data.to_csv('check6_2_입력데이터_결측제거_이상치제거.csv')

In [ ]:
# 스케일링
# 입력데이터 불러오기

# MinMaxScaling 이용
# 스케일된 데이터 저장
X = cpl_data.loc[:, '고속도로':'주차장'].values
scaled_X = MinMaxScaler().fit_transform(X)

# X1 : 스케일링된 데이터프레임
# X2 : 분석에 사용될 feature
X1 = pd.DataFrame(data = scaled_X, columns = raw_data.columns[1:])
X2 = cpl_data['grid'].reset_index().drop(columns = 'index', axis=1)

# 완성형 입력데이터 생성
cpl_data = pd.concat([X1, X2], axis=1)
cpl_data.to_csv('check6_3_입력데이터_결측제거_이상치제거_스케일링.csv')

In [ ]:
# 데이터 결합
drive_grid = pd.read_csv('/content/check4_2_운행_좌표그리드.csv', index_col = 0)
gby1 = drive_grid.pivot_table(index = 'grid', aggfunc = 'count', values = 's_lat').rename(columns = {'s_lat' : 'count'})

input_data = pd.merge(cpl_data, gby1, left_on = 'grid', right_index = True, how = 'left').fillna(0)
input_data.to_csv('check7_최종데이터.csv')


trans_gird = pd.read_csv('/content/check4_3_대중교통_좌표그리드.csv', index_col = 0)
gby1 = trans_gird.pivot_table(index = 'grid', aggfunc = 'count', values = 'lat').rename(columns = {'lat' : 'count'})

trans_data = pd.merge(cpl_data, gby1, left_on = 'grid', right_index = True, how = 'left').fillna(0)
trans_data.to_csv('check8_비교데이터_대중교통.csv')

In [ ]:
# 데이터 분할 및 샘플링

# 이진분류모델을 사용하기 위해, 'count'값이 1 이상이면 모두 1로 집계
def binary(n):
  if n == 0:
    return 0
  else:
    return 1

input_data['count'] = input_data['count'].apply(binary)

# 'count'의 비율 확인
# input_data (운행이력) _ 33921 : 1962 (불균형)
input_data['count'].value_counts()

In [ ]:
# 독립변수, 종속변수 분리
X = input_data.loc[:, '고속도로':'주차장']
y = input_data['count']

# 분리한 데이터가 불균형을 이루지 않도록, 비율에 맞게 분리
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)
for train_idx, test_idx in split.split(X, y):
  df_strat_train = input_data.loc[train_idx]
  df_strat_test = input_data.loc[test_idx]

# train, test 데이터 생성
X_train_car, y_train_car = df_strat_train.loc[:, '고속도로':'주차장'], df_strat_train['count']
X_test_car, y_test_car = df_strat_test.loc[:, '고속도로':'주차장'], df_strat_test['count']


# train, test로의 분리 시각화
sep_train = pd.concat([X_train_car, y_train_car], axis = 1)
sep_train['group'] = 'train'
sep_test = pd.concat([X_test_car, y_test_car], axis = 1)
sep_test['group'] = 'test'

sep = pd.concat([sep_train, sep_test], axis = 0)

df_group_by_0 = sep[sep['count']==0].groupby(['group']).count()
df_group_by_1 = sep[sep['count']==1].groupby(['group']).count()
label = sep['group'].unique()
label = sorted(label)
index = np.arange(len(label))

p1 = plt.bar(index, np.array(df_group_by_0)[:, 0], color='navy', alpha=0.5, width=0.45)
p2 = plt.bar(index+0.45, np.array(df_group_by_1)[:, 0], color='gray', alpha=0.5, width=0.45)
plt.xticks(index,label)
plt.legend((p1[0], p2[0]), ('0', '1'), fontsize=14)

In [ ]:
# SMOTEENN 기법을 활용하여, 다수클래스를 줄이고 소수클래스를 늘림
smt = SMOTEENN(random_state = 42)
X_car, y_car = smt.fit_resample(X_train_car, y_train_car)

# 'count'의 비율 확인
# 1 : 0 = 23215 : 19676
y_car.value_counts()
y_car.value_counts().plot(kind = 'bar', color = 'darkorange')

### 학습 진행

In [ ]:
# 필요한 라이브러리 불러오기
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold


In [ ]:
# Catboost
# 운행 데이터를 모델에 적용
cat = CatBoostClassifier(iterations=5, depth=4, learning_rate=0.49, loss_function='Logloss', verbose=True)
cat.fit(X_car, y_car)

preds_class = cat.predict(X_test_car)
preds_proba = cat.predict_proba(X_test_car)

accuracy = accuracy_score(y_test_car, preds_class)
recall = recall_score(y_test_car, preds_class)

print('class = ', preds_class)
print('proba = ', preds_proba, sep = '\n', end='\n\n')
print('accuray =', accuracy)
print('recall =', recall, end='\n\n')

In [ ]:
# XGBoost
# 운행 데이터를 모델에 적용
xgb_clf = XGBClassifier(min_child_weight = 3, max_depth = 3, sub_sample = 0.7,learning_rate = 0.2, n_estimators = 60)
xgb_clf.fit(X_car, y_car)

y_pred = xgb_clf.predict(X_test_car)

ac = accuracy_score(y_test_car, y_pred)
r = recall_score(y_test_car, y_pred)
print('accuray =', ac)
print('recall =', r)

In [ ]:
# LGBM
# 운행 데이터를 모델에 적용
lgbm_clf = LGBMClassifier(n_estimators = 400)
lgbm_clf.fit(X_car, y_car)

y_pred = lgbm_clf.predict(X_test_car)

ac = accuracy_score(y_test_car, y_pred)
r = recall_score(y_test_car, y_pred)
print('accuray =', ac)
print('recall =', r)

In [ ]:
# KNN
# 운행 데이터를 모델에 적용
classifier = KNeighborsClassifier(n_neighbors = 5)
classifier.fit(X_car, y_car)

y_pred = classifier.predict(X_test_car)

ac = accuracy_score(y_test_car, y_pred)
r = recall_score(y_test_car, y_pred)
print('accuray =', ac)
print('recall =', r)

In [ ]:
# GridSearchCV + RandomForest
# 최적 하이퍼 파라미터 찾기 _ 운행이력
params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [6, 8, 10, 12],
           'min_samples_split' : [6, 8, 10, 12]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_car, y_car)


# 최적 하이퍼 파라미터로, 운행 데이터를 모델에 적용
rf_clf1 = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 12,
                                min_samples_leaf = 6,
                                min_samples_split = 6,
                                random_state = 0,
                                n_jobs = -1)
rf_clf1.fit(X_car, y_car)
y_pred = rf_clf1.predict(X_test_car)

ac = accuracy_score(y_test_car, y_pred)
r = recall_score(y_test_car, y_pred)
print('accuray =', ac)
print('recall =', r)

In [ ]:
# Manual Search + RandomForest
# 운행 데이터를 모델에 적용
rf_clf2 = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 8,
                                min_samples_leaf = 4,
                                min_samples_split = 6,
                                random_state = 0,
                                n_jobs = -1)
rf_clf2.fit(X_car, y_car)
y_pred = rf_clf2.predict(X_test_car)

ac = accuracy_score(y_test_car, y_pred)
r = recall_score(y_test_car, y_pred)
print('accuray =', ac)
print('recall =', r)

In [ ]:
# Stacking Ensemble
def stacking_data(model, X_train, y_train, X_test, n_folds=3):     
  kfold = StratifiedKFold(n_splits = n_folds, random_state = 1004,shuffle=True)
  
  # 최종 모델에서 사용할 데이터 셋 세팅 (0 값으로)
  # 만약 shape가 (100,10)이었으면 폴드의 검증 과정 중 저장할 데이터는 (100,1)으로 함.
  train_fold_predict = np.zeros((X_train.shape[0],1))
  # test 는 X_test 값을 이용해서 매 폴드마다 예측을 하기 때문에 (100, fold갯수) 모양이다.
  # 해당 폴드마다 X_test의 예측 값을 fold 에 넣는다! meta model 이 쓸꺼임
  test_predict = np.zeros((X_test.shape[0], n_folds))
  print('model : ',model.__class__.__name__)
  
  for cnt, (train_index, valid_index) in enumerate(kfold.split(X_train,y_train)):
    X_train_ = X_train.iloc[train_index]
    y_train_ = y_train.iloc[train_index]
    X_valid = X_train.iloc[valid_index]

    # 학습
    model.fit(X_train_,y_train_)
    # 해당 폴드에서 학습된 모델에다가 검증 데이터 (X_valid)로 예측 후 저장
    train_fold_predict[valid_index,:] = model.predict(X_valid).reshape(-1,1)
    # 해당 폴드에서 생성된 모델에게 원본 테스트 데이터 (X_test)를 이용해서 예측하고 저장
    test_predict[:,cnt] = model.predict(X_test)
  
  # for 문이 끝나면 test_pred는 평균을 내서 하나로 합친다.
  test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
  
  return train_fold_predict, test_predict_mean


xgb_esti=XGBClassifier(random_state=0,base_estimator__max_depth=2,base_estimator__min_child_weight=2,base_estimator__sub_sample=0.5,n_estimators=140,learning_rate=0.3)
cat_esti=CatBoostClassifier(random_state=0,depth=5,learning_rate=0.3,n_estimators=100)
rf_esti = RandomForestClassifier(n_jobs=-1,n_estimators=100,max_depth=8)
lgbm_est=LGBMClassifier(n_jobs=-1, random_state=0,n_estimators=100)
gb_est=GradientBoostingClassifier(random_state=0)
dt_est=DecisionTreeClassifier(random_state=0)
ada_est=AdaBoostClassifier(random_state=0)


# 앙상블 수행 cat, rf, lgbm, GradientBoosting, dt, adaboost ->  xgb
cat_train, cat_test = stacking_data(cat_esti, X_car, y_car, X_test_car)
rf_train, rf_test = stacking_data(rf_esti, X_car, y_car, X_test_car)
lgbm_train, lgbm_test = stacking_data(lgbm_est, X_car, y_car, X_test_car)
gb_train, gb_test = stacking_data(gb_est, X_car, y_car, X_test_car)
dt_train, dt_test = stacking_data(dt_est, X_car, y_car, X_test_car)
ada_train, ada_test = stacking_data(ada_est, X_car, y_car, X_test_car)


# return 된 kfold 결과와 X_test 결과를 skacking
# 이 new data를 meta data로 최종 model 을 학습 시키기위해 준비
new_X_train = np.concatenate((cat_train, rf_train, lgbm_train, gb_train, dt_train, ada_train), axis=1)
new_X_test = np.concatenate((cat_test,rf_test,lgbm_test,gb_test,dt_test,ada_test), axis=1)

print('원본 : ', X_car.shape, X_test_car.shape)
print('새로운 : ', new_X_train.shape, new_X_test.shape)

# 최종 모델 훈련
xgb_esti.fit(new_X_train, y_car)
stack_pred = xgb_esti.predict(new_X_test)

print('accuracy = ', accuracy_score(y_test_car, stack_pred))
print("recall = {:.4f}".format(recall_score(y_test_car, stack_pred)))

### 모델성능 확인 및 서비스 발전

In [ ]:
# 필요한 라이브러리 불러오기
import folium
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

In [ ]:
# 선정된 모델 및 데이터로 성능검토
rf_clf2_car = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 8,
                                min_samples_leaf = 4,
                                min_samples_split = 6,
                                random_state = 0,
                                n_jobs = -1)
rf_clf2_car.fit(X_car, y_car)
y_pred = rf_clf2_car.predict(X_test_car)

ac = accuracy_score(y_test_car, y_pred)
r = recall_score(y_test_car, y_pred)
print('accuray_car =', ac)
print('recall_car =', r)

In [ ]:
# 운행 데이터의 train set에 대한 과적합 여부 확인
y_aftrain = rf_clf2_car.predict(X_train_car)
ac = accuracy_score(y_train_car, y_aftrain)
r = recall_score(y_train_car, y_aftrain)

print('accuray =', ac)
print('recall =', r)

# 결과 : 과적합 아님

In [ ]:
# 운행여부를 결정하는, 중요도가 높은 feature를 선정
feature_importance_car = rf_clf2_car.feature_importances_

# Top 중요도로 정렬하고, 쉽게 시각화하기 위해 Series 변환
visual_imp_car = pd.Series(feature_importance_car, index = X_train_car.columns)

# 중요도값 순으로 Series를 정렬
feature_top_car = visual_imp_car.sort_values(ascending=False)
plt.figure(figsize = [10, 6])
plt.title('Feature Importances')
sns.barplot(x = feature_top_car, y = feature_top_car.index, palette = "Oranges_r")

plt.show()

In [ ]:
# 운행 데이터의 중요 변수 3개 추출
top3_car = feature_top_car[:4]
top3_car

In [ ]:
# confusion matrix 생성

label=['no','yes'] # 라벨 설정
plot1 = plot_confusion_matrix(rf_clf2_car, # 분류 모델
                             X_test_car, y_test_car, # 예측 데이터와 예측값의 정답(y_true)
                             display_labels=label, # 표에 표시할 labels
                             cmap=plt.cm.Reds, # 컬러맵(plt.cm.Reds, plt.cm.rainbow 등이 있음)
                             #normalize='true') # 'true', 'pred', 'all' 중에서 지정 가능. default=None
)
plot1.ax_.set_title('Confusion Matrix')

In [ ]:
# test set 중 주차장 후보지 선정
# 다음 조건을 모두 만족하는 격자 추출

# 실제로 주차장이 없는 곳 ('count' = 0)
# 주차장이 있다고 예측된 곳 ('pred_car' = 1)
# 대중교통 역이 실제로 없는 곳 ('trans' = 0)

trans = pd.read_csv('check8_비교데이터_대중교통.csv', index_col=0)

pred_car = rf_clf2_car.predict(X_test_car)
pred_car = pd.Series(pred_car, index = y_test_car.index)
trans = trans[['grid', 'count']]

cand = X_test_car
cand = pd.concat([cand, y_test_car, pred_car], axis=1).rename(columns = {0 : 'pred_car'})
cand = pd.merge(cand, trans, left_index = True, right_on = 'grid', how = 'left')

cand.index = cand['grid']
cand = cand.rename(columns = {'count_x' : 'count', 'count_y' : 'trans'})
cand['trans'] = cand['trans'].fillna(0).apply(binary)

In [ ]:
# test set 중 주차장 후보지 격자 추출
park = cand[(cand['count'] == 0) & (cand['pred_car'] == 1) & (cand['trans'] == 0)]

# 1141개의 후보

In [ ]:
# 전체 후보 중 분석할 2개의 위치 임의추출
import random
random.seed(0)
analy = random.sample(list(park.index), k=2)

# 해당 격자의 위치정보 추출
center = pd.read_csv('check3_1_그리드_중심정보.csv', index_col = 0)
polygon = pd.read_csv('check3_2_그리드_좌표정보.csv', index_col = 0)

center = center.loc[analy, :]
polygon = polygon.loc[analy, :]

In [ ]:
# 분석할 샘플 시각화
map_analy = folium.Map(location = [36, 128])
for i in analy:
  folium.PolyLine(locations = np.append(np.array(polygon.loc[i].tolist()), np.array(polygon.loc[i].tolist())[:2]).reshape(-1, 2), fill = True, tooltip = 'Polyline').add_to(map_analy)

map_analy